`https://python.langchain.com/docs/tutorials/rag/`

In [1]:
from langchain.schema import Document   # whichever Document import you use
import gzip, pickle
from langchain_text_splitters import RecursiveCharacterTextSplitter
from helper.academicCloudEmbeddings import AcademicCloudEmbeddings
from langchain.vectorstores import FAISS
import streamlit as st
from langchain.retrievers import ParentDocumentRetriever
from langchain.docstore import InMemoryDocstore

# Indexing
## 1. Load the data
*In our case the data needs to be crawled first. See `crawl.ipynb`. There we are storing the documents in a pickle file we can load here*

In [2]:
with gzip.open("docs.pkl.gz", "rb") as f:
    docs = pickle.load(f)

# 2.  give every parent document a stable id  (one-liner)
for i, d in enumerate(docs):
    d.metadata["doc_id"] = f"doc_{i}"            # <- save this id

# Contextual Retrieval (NEW)

In [3]:
from openai import OpenAI, RateLimitError
from copy import deepcopy            # or import Document directly if you need to
# docs is your original list[Document]
from pathlib import Path
import time

client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"])

# Path to text file
file_path = Path("docs.txt")

# Read all text into the variable `text`
WHOLE_DOCUMENT = file_path.read_text(encoding="utf-8")

def build_prompt(chunk: str) -> str:
    return f"""<document>
    {WHOLE_DOCUMENT}
    </document>
    <chunk>
    {chunk}
    </chunk>
    Bitte gib einen kurzen, prägnanten Kontext an, um diesen Abschnitt im Gesamtdokument einzuordnen und die Suche nach diesem Abschnitt zu verbessern. Antworte nur mit dem prägnanten Kontext und nichts anderem. Dieser Kontext soll in einer späteren RAG Pipeline hilfreich sein. Der Kontext den du schreibst MUSS helfen, den Abschnitt zu finden, wenn jemand nach dem Abschnitt sucht. Schreibe **einen Satz (max 50 Wörter)**, der den Chunk für eine RAG-Suche einordnet.
    • Nutze nur Informationen, die im Chunk vorkommen.
    • Vermeide Interpretationen oder zusätzliche Details.
    • Benutze Keywords nur, wenn in diesem Chunk auch auf diese eingegangen wird. Nicht, nur wiel sie im Chunk erwähnt werden."""

for doc in docs:
    while True:
        try:
            context = client.responses.create(
                model="gpt-4.1-nano",
                input=build_prompt(doc.page_content)
            ).output_text.strip()
            doc.metadata["context"] = context
            doc.page_content = context + "\n" + doc.page_content
            break
        except RateLimitError:
            time.sleep(10)  # Wait and retry

docs[1]

Document(metadata={'url': 'https://wiki.student.uni-goettingen.de/support/account/benutzerordnung', 'doc_id': 'doc_1', 'context': 'Hinweis auf die Nutzerordnung für den studentischen Account an der Georg-August-Universität Göttingen, inklusive Warnungen zu Apps Dritter und Datenschutzbestimmungen.'}, page_content='Hinweis auf die Nutzerordnung für den studentischen Account an der Georg-August-Universität Göttingen, inklusive Warnungen zu Apps Dritter und Datenschutzbestimmungen.\nsupport:account:benutzerordnung\nLiebe Studierende, \nmöglicherweise haben Sie bereits von Apps gehört, die nicht von der Universität Göttingen selbst angeboten werden, die es Ihnen aber ermöglichen, Dienste der Universität (bspw. FlexNow) darüber zu nutzen. Dabei werden Sie dazu aufgefordert, Zugangsdaten Ihres Accounts in eine solche App einzutragen.  \nDie Abteilung IT und das Rechenzentrum der Universität (GWDG) möchten Sie hiermit ausdrücklich vor der Nutzung solcher Apps oder vergleichbarer Dienste ander

## 2. Split the loaded data
We are splitting large documents into smaller chunks for indexing the data and passing it into a model. Large chunks would be worse for search

In [4]:
# splitten – jede URL bleibt als metadata erhalten
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],
)
child_docs = splitter.split_documents(docs)

## 3. Store
We are storing the created chunks:
1. creating embeddings using the GWDG model
2. storing in a FAISS store which can be saved locally to use later. That way we don't need to create the store every time we want to start the app

In [5]:
# Embeddings und FAISS
embedder = AcademicCloudEmbeddings(
    api_key=st.secrets["GWDG_API_KEY"],
    url=st.secrets["BASE_URL_EMBEDDINGS"],
)
child_vs = FAISS.from_documents(child_docs, embedder)
child_vs.save_local("faiss_child_index")

In [6]:
from langchain_core.stores import InMemoryStore

# store the *parents* in an in-memory doc-store and pickle it
parent_store = InMemoryStore()
parent_store.mset([(d.metadata["doc_id"], d) for d in docs])

with open("parent_store.pkl", "wb") as f:
    pickle.dump(parent_store, f)

In [7]:
# test
with open("parent_store.pkl", "rb") as f:
    parent_store = pickle.load(f)

retriever = ParentDocumentRetriever(
    vectorstore    = child_vs,   # FAISS with your child chunks
    docstore       = parent_store,   # ✔︎ now a BaseStore subclass
    child_splitter = splitter,
    search_kwargs  = {"k": 4},
)
print(retriever.get_relevant_documents("Eduroam Iphone")[0].page_content)

/var/folders/pp/8qh_wz3n6qd12281djkgtrhh0000gn/T/ipykernel_47195/4187668320.py:11: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(retriever.get_relevant_documents("Eduroam Iphone")[0].page_content)


ConnectTimeout: HTTPSConnectionPool(host='chat-ai.academiccloud.de', port=443): Max retries exceeded with url: /v1/embeddings (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10863b550>, 'Connection to chat-ai.academiccloud.de timed out. (connect timeout=None)'))